<a href="https://colab.research.google.com/github/HaeSeon/nlp-ner/blob/main/%5Bbert_ner%5DCONLL_data_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting

DATA : conll2003


In [1]:
!git clone https://github.com/kamalkraj/BERT-NER.git

Cloning into 'BERT-NER'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 249 (delta 0), reused 0 (delta 0), pack-reused 246
Receiving objects: 100% (249/249), 1.67 MiB | 21.14 MiB/s, done.
Resolving deltas: 100% (125/125), done.


In [2]:
cd BERT-NER/

/content/BERT-NER


In [3]:
# bERT-NER file
%ls -l

total 100
-rw-r--r-- 1 root root   470 May 18 06:30 api.py
-rw-r--r-- 1 root root  4674 May 18 06:30 bert.py
drwxr-xr-x 3 root root  4096 May 18 06:30 cpp-app/
drwxr-xr-x 2 root root  4096 May 18 06:30 data/
drwxr-xr-x 2 root root  4096 May 18 06:30 img/
-rw-r--r-- 1 root root 34523 May 18 06:30 LICENSE.txt
-rw-r--r-- 1 root root  4777 May 18 06:30 README.md
-rw-r--r-- 1 root root   173 May 18 06:30 requirements.txt
-rw-r--r-- 1 root root 27023 May 18 06:30 run_ner.py


In [4]:
# install package
!pip3 install -r requirements.txt

     |████████████████████████████████| 184kB 8.8MB/s 
     |████████████████████████████████| 748.9MB 11kB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 1.5MB 35.3MB/s 
     |████████████████████████████████| 102kB 12.1MB/s 
     |████████████████████████████████| 102kB 12.9MB/s 
     |████████████████████████████████| 901kB 37.3MB/s 
     |████████████████████████████████| 1.2MB 33.1MB/s 
     |████████████████████████████████| 7.5MB 36.0MB/s 
     |████████████████████████████████| 81kB 9.0MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp37-none-any.whl size=1449906 sha256=3ca605d81744f82c2a143ded7fe70fff9d70d75e67f14e0710dc6a8cb0bcd2b6
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
  Created wheel for boto3: filename=boto3-1.17.74-py2.py3-none-any.whl size=128921 sha256=7a325e47e3dae3e722f8119305b9b10766ec5f6b84083ff8e3783cfca562cf57
  Stored in directory: /root/.cach

# Fine-tuning

finetuning 또는 훈련을 하려면 run_ner.py 실행한다.

아래와 같은 arguments를 전달해야한다. 
* data_dir : 사용할 date set
* -bert_model : Hugging Face transformers의 pretrain bert base 모델을 다운로드하기 위해 사용
* --task_name : 수행할 작업. 여기서는 ner
* --output_dir : fine-tuned 모델을 저장할 위치
* max_seq_length
* do_train
* --num_train_epoch
* --do_eval
* --warmup_proportion



In [5]:
!python run_ner.py --data_dir=data/ --bert_model=bert-base-cased --task_name=ner --output_dir=out_ner --max_seq_length=128 --do_train --num_train_epochs 3 --do_eval --warmup_proportion=0.1

05/18/2021 06:33:16 - INFO - __main__ -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
05/18/2021 06:33:16 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt not found in cache or force_download set to True, downloading to /tmp/tmpbs849sqf
100% 213450/213450 [00:00<00:00, 836779.14B/s]
05/18/2021 06:33:17 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmpbs849sqf to cache at /root/.cache/torch/pytorch_transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
05/18/2021 06:33:17 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
05/18/2021 06:33:17 - INFO - pytorch_transformers.file_utils -   removing te

In [6]:
pwd

'/content/BERT-NER'

# Prediction

**overwrite bert.py file**

entity derect, entity types를 더 잘 나타내기 위해서 덮어쓴다. 


In [7]:
%%writefile bert.py
"""BERT NER Inference."""

from __future__ import absolute_import, division, print_function

import json
import os

import torch
import torch.nn.functional as F
from nltk import word_tokenize
from pytorch_transformers import (BertConfig, BertForTokenClassification,
                                  BertTokenizer)


class BertNer(BertForTokenClassification):

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, valid_ids=None):
        sequence_output = self.bert(input_ids, token_type_ids, attention_mask, head_mask=None)[0]
        batch_size,max_len,feat_dim = sequence_output.shape
        valid_output = torch.zeros(batch_size,max_len,feat_dim,dtype=torch.float32,device='cuda' if torch.cuda.is_available() else 'cpu')
        for i in range(batch_size):
            jj = -1
            for j in range(max_len):
                    if valid_ids[i][j].item() == 1:
                        jj += 1
                        valid_output[i][jj] = sequence_output[i][j]
        sequence_output = self.dropout(valid_output)
        logits = self.classifier(sequence_output)
        return logits

class Ner:

    def __init__(self,model_dir: str):
        self.model , self.tokenizer, self.model_config = self.load_model(model_dir)
        self.label_map = self.model_config["label_map"]
        self.max_seq_length = self.model_config["max_seq_length"]
        self.label_map = {int(k):v for k,v in self.label_map.items()}
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = self.model.to(self.device)
        self.model.eval()

    def load_model(self, model_dir: str, model_config: str = "model_config.json"):
        model_config = os.path.join(model_dir,model_config)
        model_config = json.load(open(model_config))
        model = BertNer.from_pretrained(model_dir)
        tokenizer = BertTokenizer.from_pretrained(model_dir, do_lower_case=model_config["do_lower"])
        return model, tokenizer, model_config

    def tokenize(self, text: str):
        """ tokenize input"""
        words = word_tokenize(text)
        tokens = []
        valid_positions = []
        for i,word in enumerate(words):
            token = self.tokenizer.tokenize(word)
            tokens.extend(token)
            for i in range(len(token)):
                if i == 0:
                    valid_positions.append(1)
                else:
                    valid_positions.append(0)
        return tokens, valid_positions

    def preprocess(self, text: str):
        """ preprocess """
        tokens, valid_positions = self.tokenize(text)
        ## insert "[CLS]"
        tokens.insert(0,"[CLS]")
        valid_positions.insert(0,1)
        ## insert "[SEP]"
        tokens.append("[SEP]")
        valid_positions.append(1)
        segment_ids = []
        for i in range(len(tokens)):
            segment_ids.append(0)
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_ids)
        while len(input_ids) < self.max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)
            valid_positions.append(0)
        return input_ids,input_mask,segment_ids,valid_positions

    def predict(self, text: str):
        input_ids,input_mask,segment_ids,valid_ids = self.preprocess(text)
        input_ids = torch.tensor([input_ids],dtype=torch.long,device=self.device)
        input_mask = torch.tensor([input_mask],dtype=torch.long,device=self.device)
        segment_ids = torch.tensor([segment_ids],dtype=torch.long,device=self.device)
        valid_ids = torch.tensor([valid_ids],dtype=torch.long,device=self.device)
        with torch.no_grad():
            logits = self.model(input_ids, segment_ids, input_mask,valid_ids)
        logits = F.softmax(logits,dim=2)
        logits_label = torch.argmax(logits,dim=2)
        logits_label = logits_label.detach().cpu().numpy().tolist()[0]

        logits_confidence = [values[label].item() for values,label in zip(logits[0],logits_label)]

        logits = []
        pos = 0
        for index,mask in enumerate(valid_ids[0]):
            if index == 0:
                continue
            if mask == 1:
                logits.append((logits_label[index-pos],logits_confidence[index-pos]))
            else:
                pos += 1
        logits.pop()

        labels = [(self.label_map[label],confidence) for label,confidence in logits]
        words = word_tokenize(text)
        assert len(labels) == len(words)

        Person = []
        Location = []
        Organization = []
        Miscelleneous = []

        for word, (label, confidence) in zip(words, labels):
            if label=="B-PER" or label=="I-PER":
                Person.append(word)
            elif label=="B-LOC" or label=="I-LOC":
                Location.append(word)
            elif label=="B-ORG" or label=="I-ORG":
                Organization.append(word)
            elif label=="B-MISC" or label=="I-MISC":
                Miscelleneous.append(word)
            else:
                output = None

        output = []
        for word, (label, confidence) in zip(words, labels):      
            if label == "B-PER":
                output.append(' '.join(Person) + ": Person")
            if label=="B-LOC":
                output.append(' '.join(Location) + ": Location")
            if label=="B-MISC":
                output.append(' '.join(Miscelleneous) + ": Miscelleneous Entity")
            if label=="B-ORG":
                output.append(' '.join(Organization) + ": Organization")
                
        return output


Overwriting bert.py


In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Final output

* line1 : Ner 클래스를 bert.py 파일로부터 불러온다. Ner 클래스는 위에서 진행한 우리의 fine-tuned 모델을 초기화한다. 

* line2 : fine-tuned 모델을 model 변수에 저장한다. 

* line3,4 : 예측할 텍스트를 입력한다. 

* line 5 : Ner 클래스의 predict 함수를 사용하여 output에 저장한다. output에는 detected entities의 리스트가 저장된다. 

* line 6,7 : output 출력

In [9]:
from bert import Ner
model = Ner("out_ner/")

text= "The U.S. President Donald Trump came to visit Ahmedabad first time at Motera Stadium with our Prime Minister Narendra Modi in February 2020."
print("Text to predict Entity:", text)

output = model.predict(text)
for prediction in output:
    print(prediction)

Text to predict Entity: The U.S. President Donald Trump came to visit Ahmedabad first time at Motera Stadium with our Prime Minister Narendra Modi in February 2020.
U.S. Ahmedabad Motera Stadium: Location
Donald Trump Narendra Modi: Person
U.S. Ahmedabad Motera Stadium: Location
U.S. Ahmedabad Motera Stadium: Location
Donald Trump Narendra Modi: Person
